In [ ]:
# default_exp clone_counters

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# export
from functools import partial
from glob import glob
from typing import Callable

import dask.array as da
import dask.dataframe as dd
import numpy as np
import pandas as pd
import xarray as xr
from skimage import measure

from py_clone_detective.utils import (
    add_scale_regionprops_table_area_measurements,
    calculate_corresponding_labels,
    calculate_overlap,
    check_channels_input_suitable_and_return_channels,
    determine_labels_across_other_images_using_centroids,
    extend_region_properties_list,
    get_all_labeled_clones_unmerged_and_merged,
    img_path_to_xarr,
    last2dims,
    lazy_props,
    reorder_df_to_put_ch_info_first,
    update_1st_coord_and_dim_of_xarr,
)

# CloneCounter Classes

## Parent Class

In [ ]:
# export
class CloneCounter:
    def __init__(
        self,
        exp_name: str,
        img_name_regex: str,
        pixel_size: float,
        tot_seg_ch: str = "C0",
    ):
        self.exp_name = exp_name
        self.img_name_regex = img_name_regex
        self.pixel_size = pixel_size
        self.tot_seg_ch = tot_seg_ch

    def add_images(self, **channel_path_globs):
        return img_path_to_xarr(
            self.img_name_regex,
            self.pixel_size,
            ch_name_for_first_dim="img_channels",
            **channel_path_globs,
        )

    def add_segmentations(
        self,
        additional_func_to_map: Callable = None,
        ad_func_kwargs: dict = None,
        **channel_path_globs,
    ):
        segmentations = img_path_to_xarr(
            self.img_name_regex,
            self.pixel_size,
            ch_name_for_first_dim="seg_channels",
            **channel_path_globs,
        )

        if additional_func_to_map is not None:
            segmentations.data = segmentations.data.map_blocks(
                additional_func_to_map, **ad_func_kwargs, dtype=np.uint16
            )

        segmentations.data = segmentations.data.map_blocks(
            last2dims(partial(measure.label)), dtype=np.uint16
        )
        return segmentations

    def combine_C0_overlaps_and_measurements(self):
        ov_df = (
            self.results_overlaps.pivot(
                index=["img_name", "C0_labels"],
                columns=["colocalisation_ch"],
                values="is_in_label",
            )
            .query("C0_labels != 0")
            .copy()
        )
        sk_df = self.results_measurements.query("seg_channel == 'C0'").set_index(
            ["segmentation_img_name", "label"]
        )
        sk_df.index.rename(["img_name", "C0_labels"], inplace=True)
        return pd.merge(ov_df, sk_df, left_index=True, right_index=True)

    def determine_seg_img_channel_pairs(
        self, seg_channels: list = None, img_channels: list = None
    ):
        seg_channels = check_channels_input_suitable_and_return_channels(
            channels=seg_channels,
            available_channels=self.image_data.seg_channels.values.tolist(),
        )

        img_channels = check_channels_input_suitable_and_return_channels(
            channels=img_channels,
            available_channels=self.image_data.img_channels.values.tolist(),
        )

        seg_img_channel_pairs = pd.DataFrame()
        seg_img_channel_pairs["image_channel"] = pd.Series(img_channels)
        seg_img_channel_pairs["segmentation_channel"] = pd.Series(seg_channels)
        self.seg_img_channel_pairs = seg_img_channel_pairs.fillna(method="ffill")[
            ["segmentation_channel", "image_channel"]
        ]

    def make_measurements(
        self,
        seg_channels: list = None,
        img_channels: list = None,
        extra_properties: list = None,
        **kwargs,
    ):

        self.determine_seg_img_channel_pairs(seg_channels, img_channels)

        properties = extend_region_properties_list(extra_properties)

        results = list()
        for _, seg_ch, img_ch in self.seg_img_channel_pairs.itertuples():
            for seg, img in zip(
                self.image_data["segmentations"].loc[seg_ch],
                self.image_data["images"].loc[img_ch],
            ):
                results.append(
                    lazy_props(
                        seg.data,
                        img.data,
                        seg.seg_channels.item(),
                        img.img_channels.item(),
                        seg.img_name.item(),
                        img.img_name.item(),
                        properties,
                        **kwargs,
                    )
                )

        df = dd.from_delayed(results).compute()
        df = add_scale_regionprops_table_area_measurements(df, self.pixel_size)
        self.results_measurements = reorder_df_to_put_ch_info_first(df)
        self._determine_max_seg_label_levels()

    def _determine_max_seg_label_levels(self):
        self.tot_seg_ch_max_labels = (
            self.image_data["segmentations"]
            .loc[self.tot_seg_ch]
            .data.map_blocks(
                lambda x: np.unique(x).shape[0], drop_axis=(1, 2), dtype=np.uint16,
            )
            .compute()
            .max()
        )

    def _create_df_from_arr(self, arr):
        return (
            xr.DataArray(
                np.moveaxis(arr, 1, 0),
                coords=(
                    self.image_data["segmentations"].coords["seg_channels"][1:],
                    self.image_data["segmentations"].coords["img_name"],
                    np.arange(self.max_seg_label_levels["C0"]),
                ),
                dims=("colocalisation_ch", "img_name", "C0_labels",),
            )
            .to_dataframe("is_in_label")
            .reset_index()
            .dropna()
        )

    def measure_overlap(self):
        self._determine_max_seg_label_levels("C0")
        arr = (
            self.image_data["segmentations"]
            .data.map_blocks(
                calculate_overlap,
                drop_axis=[0],
                dtype=np.float64,
                num_of_segs=self.image_data["segmentations"].shape[0],
                preallocate_value=self.max_seg_label_levels["C0"],
            )
            .compute()
        )

        df = self._create_df_from_arr(arr)
        df["is_in_label"] = df["is_in_label"].astype(np.uint16)
        self.results_overlaps = df[
            ["img_name", "C0_labels", "colocalisation_ch", "is_in_label"]
        ]

    def clones_to_keep_as_dict(self, query_for_pd: str):
        return (
            self.results_measurements.query(query_for_pd)
            .groupby("intensity_img_name")
            .agg({"label": lambda x: list(x)})["label"]
            .to_dict()
        )

    def get_centroids_list(self):
        df = self.results_measurements.query(
            "intensity_img_channel == @self.tot_seg_ch"
        )
        centroids_list = list()
        for img_name in df["intensity_img_name"].unique():
            centroids_list.append(
                (
                    df.query("intensity_img_name == @img_name")
                    .loc[:, ["centroid-0", "centroid-1"]]
                    .values.astype(int)
                )
            )
        return centroids_list

    def add_clones_and_neighbouring_labels(
        self,
        query_for_pd: str = 'intensity_img_channel == "C1" & mean_intensity > 1000',
        name_for_query: str = "filt_C1_intensity",
    ):
        clone_coords, clone_dims = update_1st_coord_and_dim_of_xarr(
            self.image_data["images"],
            new_coord=[
                f"extended_tot_seg_labels",
                f"clone",
                f"total_neighbour_counts",
                f"inside_clone_neighbour_counts",
                f"outside_clone_neighbour_counts",
            ],
            new_dim="extended_labels_neighbour_counts",
        )

        clones_to_keep = self.clones_to_keep_as_dict(query_for_pd)

        new_label_imgs = get_all_labeled_clones_unmerged_and_merged(
            self.image_data["segmentations"].loc[self.tot_seg_ch], clones_to_keep
        )

        return xr.DataArray(
            data=new_label_imgs,
            coords=clone_coords,
            dims=clone_dims,
            attrs={f"{self.tot_seg_ch}_labels_kept_query": query_for_pd},
        )

    def colabels_to_df(self, colabels, name_for_query):
        return (
            xr.DataArray(
                colabels,
                coords=(
                    self.image_data[name_for_query].coords[
                        "extended_labels_neighbour_counts"
                    ],
                    foo.image_data[name_for_query].coords["img_name"],
                    range(colabels.shape[2]),
                ),
                dims=("extended_labels_neighbour_counts", "img_name", "labels"),
            )
            .to_dataframe("colabel")
            .reset_index()
            .dropna()
            .pivot(
                index=["img_name", "labels"],
                columns=["extended_labels_neighbour_counts"],
                values="colabel",
            )
            .astype(np.uint16)
        )

    def measure_clones_and_neighbouring_labels(self, name_for_query):
        self.get_centroids_list()
        colabels = calculate_corresponding_labels(
            self.image_data[name_for_query].data,
            self.get_centroids_list(),
            self.image_data[name_for_query].shape[0],
            foo.tot_seg_ch_max_labels,
        )

        self.results_clones_and_neighbour_counts = self.colabels_to_df(
            colabels, name_for_query
        )

## CloneCounter subclasses

In [ ]:
# export
class LazyCloneCounter(CloneCounter):
    def __init__(self, exp_name: str, img_name_regex: str, pixel_size: float):
        super().__init__(exp_name, img_name_regex, pixel_size)

    def add_images(self, **channel_path_globs):
        self.image_data = xr.Dataset(
            {"images": super().add_images(**channel_path_globs)}
        )

    def add_segmentations(
        self,
        additional_func_to_map: Callable = None,
        ad_func_kwargs: dict = None,
        **channel_path_globs
    ):
        self.image_data["segmentations"] = super().add_segmentations(
            additional_func_to_map, ad_func_kwargs, **channel_path_globs
        )

    def add_clones_and_neighbouring_labels(
        self,
        query_for_pd: str = 'intensity_img_channel == "C1" & mean_intensity > 1000',
        name_for_query: str = "filt_C1_intensity",
    ):
        self.image_data[name_for_query] = super().add_clones_and_neighbouring_labels(
            query_for_pd, name_for_query
        )

In [ ]:
# export
class PersistentCloneCounter(CloneCounter):
    def __init__(self, exp_name: str, img_name_regex: str, pixel_size: float):
        super().__init__(exp_name, img_name_regex, pixel_size)

    def add_images(self, **channel_path_globs):
        self.image_data = xr.Dataset(
            {"images": super().add_images(**channel_path_globs)}
        ).persist()

    def add_segmentations(
        self,
        additional_func_to_map: Callable = None,
        ad_func_kwargs: dict = None,
        **channel_path_globs,
    ):
        self.image_data["segmentations"] = (
            super()
            .add_segmentations(
                additional_func_to_map, ad_func_kwargs, **channel_path_globs
            )
            .persist()
        )

    def add_clones_and_neighbouring_labels(
        self,
        query_for_pd: str = 'intensity_img_channel == "C1" & mean_intensity > 1000',
        name_for_query: str = "filt_C1_intensity",
    ):
        self.image_data[name_for_query] = (
            super()
            .add_clones_and_neighbouring_labels(query_for_pd, name_for_query)
            .persist()
        )

In [ ]:
# hide
from dask.distributed import Client

c = Client()
c

Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:56508,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:56527,Total threads: 2
Dashboard: http://127.0.0.1:56528/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:56513,


## Example using LazyCloneCounter with measure_overlap

In [ ]:
foo = LazyCloneCounter("Marcm2a_E7F1", r"a\dg\d\dp\d", 0.275)

foo.add_images(
    C0="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C0/C0_imgs/*.tif*",
    C1="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C1/C1_imgs/*.tif*",
    C2="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C2/C2_imgs/*.tif*",
    C3="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C3/C3_imgs/*.tif*",
)

foo.add_segmentations(
    C0="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C0/C0_label_imgs_combined_C3/*.tif*",
    C1="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C1/C1_binaries/*.tif*",
    C2="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C2/C2_label_imgs_v2/*.tif*",
    C3="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C3/C3_label_imgs/*.tif*",
)
foo.make_measurements(extra_properties=["convex_area"],)
foo.measure_overlap()
foo.combine_C0_overlaps_and_measurements()

## Example using LazyCloneCounter with add_clones_and_neighbouring_labels

In [ ]:
from skimage import morphology

In [ ]:
foo = PersistentCloneCounter("Marcm2a_E7F1", r"a\dg\d\dp\d", 0.275)

foo.add_images(
    C0="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C0/C0_imgs/*.tif*",
    C1="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C1/C1_imgs/*.tif*",
    C2="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C2/C2_imgs/*.tif*",
    C3="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C3/C3_imgs/*.tif*",
)

In [ ]:
foo.add_segmentations(
    morphology.remove_small_objects,
    ad_func_kwargs={"min_size": 49},
    C0="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C0/C0_label_imgs_combined_C3/*.tif*",
)

In [ ]:
foo.make_measurements(extra_properties=["convex_area"],)

In [ ]:
foo.add_clones_and_neighbouring_labels(
    query_for_pd='intensity_img_channel == "C1" & mean_intensity > 1000',
    name_for_query="filt_C1_intensity",
)

foo.measure_clones_and_neighbouring_labels(name_for_query="filt_C1_intensity")

In [ ]:
import napari

In [ ]:
view = napari.Viewer()
view.add_labels(foo.image_data['filt_C1_intensity'].data)

<Labels layer 'Labels' at 0x7fcb7c5136a0>

In [ ]:
foo.image_data['filt_C1_intensity']

,Array,Chunk
Bytes,469.28 MiB,6.26 MiB
Shape,"(5, 75, 810, 810)","(5, 1, 810, 810)"
Count,75 Tasks,75 Chunks
Type,uint16,numpy.ndarray


In [ ]:
df = foo.results_clones_and_neighbour_counts.reset_index().copy()

In [ ]:
df

extended_labels_neighbour_counts,img_name,labels,clone,extended_tot_seg_labels,inside_clone_neighbour_counts,outside_clone_neighbour_counts,total_neighbour_counts
0,a1g01p1,0,0,0,0,0,0
1,a1g01p1,1,0,0,0,0,0
2,a1g01p1,2,0,0,0,0,0
3,a1g01p1,3,0,0,0,0,0
4,a1g01p1,4,0,0,0,0,0
...,...,...,...,...,...,...,...
20197,a2g13p3,236,0,0,0,0,0
20198,a2g13p3,237,0,0,0,0,0
20199,a2g13p3,238,0,0,0,0,0
20200,a2g13p3,239,0,0,0,0,0


In [ ]:
df  = df.query("extended_tot_seg_labels != 0")[
    [ "extended_tot_seg_labels",
        "img_name",
        "clone",
        "total_neighbour_counts",
        "inside_clone_neighbour_counts",
        "outside_clone_neighbour_counts",
    ]
]

In [ ]:
bar = foo.results_measurements.copy()

In [ ]:
pd.merge(bar, df, how='inner', left_on=['intensity_img_name', 'label'], right_on=['img_name', 'extended_tot_seg_labels']).drop(columns = ['img_name', 'extended_tot_seg_labels'])

,seg_channel,intensity_img_channel,segmentation_img_name,intensity_img_name,label,area,mean_intensity,centroid-0,centroid-1,convex_area,area_um2,convex_area_um2,clone,total_neighbour_counts,inside_clone_neighbour_counts,outside_clone_neighbour_counts
0,C0,C0,a1g01p1,a1g01p1,12,1094,2103.588665,43.901280,89.606947,1116,82.733750,84.397500,0,3,0,2
1,C0,C1,a1g01p1,a1g01p1,12,1094,60.372029,43.901280,89.606947,1116,82.733750,84.397500,0,3,0,2
2,C0,C2,a1g01p1,a1g01p1,12,1094,2209.449726,43.901280,89.606947,1116,82.733750,84.397500,0,3,0,2
3,C0,C3,a1g01p1,a1g01p1,12,1094,73.462523,43.901280,89.606947,1116,82.733750,84.397500,0,3,0,2
4,C0,C0,a1g01p1,a1g01p1,13,1123,3414.680321,51.479964,362.270703,1159,84.926875,87.649375,0,3,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67151,C0,C3,a2g13p3,a2g13p3,232,723,107.864454,772.175657,475.138313,749,54.676875,56.643125,0,3,0,3
67152,C0,C0,a2g13p3,a2g13p3,233,253,1257.762846,771.916996,675.988142,275,19.133125,20.796875,0,2,0,2
67153,C0,C1,a2g13p3,a2g13p3,233,253,32.418972,771.916996,675.988142,275,19.133125,20.796875,0,2,0,2
67154,C0,C2,a2g13p3,a2g13p3,233,253,742.395257,771.916996,675.988142,275,19.133125,20.796875,0,2,0,2


In [ ]:
bar